In [34]:
import pandas as pd
import numpy as np
import os
import json
import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tflearn
import nltk
from nltk.stem.lancaster import LancasterStemmer

In [10]:
df = pd.read_csv(os.getcwd()+'/../Data/NLPData.csv')
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
df.head(10)

,TAGS,PATTERNS,RESPONSES
0,GREETINGS,"Hi Hello, How are you, Is there anyone, Good M...","Hey there, friend! I'm InvestoBot, the chatbot..."
1,NAME,"I am <name>, my name is <name> , its <name>, I...",Thank you <name>! Do you have an account with us?
2,ACCOUNT_NUMBER,"Do I need to have an account ? , I have an acc...",Kindly enter your 9 digit account number to as...
3,ACCOUNT_DETAILS_VALIDITY,"How does my portfolio look ?, Can you help me ...",Please wait until I get your details. This is ...
4,ACCOUNT_DETAILS_INVALIDITY,"Sorry, I am unable to find my account number ,...","Don't worry, I am here to help! I can help you..."
5,ACCOUNT_DOES_NOT_EXIST,"I don't have my account yet, I need some help,...",Please answer the 12 Qs in the form and my adv...
6,INVEST,"I need your help with investments , How can yo...","Would you like to optimize your investments?, ..."
7,STOCK_SUGGESTIONS,I am looking for some options on Stock Trading...,"For Stocks, I can make suggestions depending o..."
8,MORE_HELP,"I need some more resolution on the trading, I ...","My advisor will reach out to you shortly., Ok ..."
9,THANKS,"Have a great day, it was great to chat, Thanks...",Thank you for chatting with me.


In [11]:
newdf = pd.DataFrame(columns=['TAGS','PATTERNS'])
index = 0
for i in range(len(df)):
    arr = df['PATTERNS'][i].split(',')
    for a in arr:
        newdf.set_value(index, 'TAGS', df['TAGS'][i])
        newdf.set_value(index, 'PATTERNS', a)
        index += 1
newdf

F:\anaconda\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  
F:\anaconda\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys


,TAGS,PATTERNS
0,GREETINGS,Hi Hello
1,GREETINGS,How are you
2,GREETINGS,Is there anyone
3,GREETINGS,Good Morning
4,GREETINGS,I need some help
5,GREETINGS,Howdy !!
6,GREETINGS,Hey
7,GREETINGS,I have a question regarding investments
8,GREETINGS,hi
9,GREETINGS,Good Afternoon ! How is your day going?


In [12]:
class PreProcessMessage():

    def __init__(self):
        pass

    def preprocess_word(self, word):
        # Remove punctuation
        word = word.strip('\'"?!,.():;')
        # Convert more than 2 letter repetitions to 2 letter
        # funnnnny --> funny
        word = re.sub(r'(.)\1+', r'\1\1', word)
        # Remove - & '
        word = re.sub(r'(-|\')', '', word)
        return word


    def is_valid_word(self, word):
        # Check if word begins with an alphabet
        return (re.search(r'^[a-zA-Z][a-z0-9A-Z\._]*$', word) is not None)


    def handle_emojis(self, message):
        # Smile -- :), : ), :-), (:, ( :, (-:, :')
        message = re.sub(r'(:\s?\)|:-\)|\(\s?:|\(-:|:\'\))', ' EMO_POS ', message)
        # Laugh -- :D, : D, :-D, xD, x-D, XD, X-D
        message = re.sub(r'(:\s?d|:-d|x-?d|x-?d)', ' EMO_POS ', message)
        # Love -- <3, :*
        message = re.sub(r'(<3|:\*)', ' EMO_POS ', message)
        # Wink -- ;-), ;), ;-D, ;D, (;,  (-;
        message = re.sub(r'(;-?\)|;-?d|\(-?;)', ' EMO_POS ', message)
        # Sad -- :-(, : (, :(, ):, )-:
        message = re.sub(r'(:\s?\(|:-\(|\)\s?:|\)-:)', ' EMO_NEG ', message)
        # Cry -- :,(, :'(, :"(
        message = re.sub(r'(:,\(|:\'\(|:"\()', ' EMO_NEG ', message)
        return message

    def preprocess_message(self, message):
        processed_message = []
        # Convert to lower case
        message = message.lower()
        # Replaces URLs with the word URL
        message = re.sub(r'((www\.[\S]+)|(https?://[\S]+))', ' URL ', message)
        # Replace 2+ dots with space
        message = re.sub(r'\.{2,}', ' ', message)
        # Strip space, " and ' from message
        message = message.strip(' "\'')
        # Replace emojis with either EMO_POS or EMO_NEG
        message = self.handle_emojis(message)
        # Replace multiple spaces with a single space
        message = re.sub(r'\s+', ' ', message)
        words = message.split()

        for word in words:
            word = self.preprocess_word(word)
            if self.is_valid_word(word):
                processed_message.append(word)
        return ' '.join(processed_message)

    def transformCleanedMessage(self, message):
        tfv=TfidfVectorizer(min_df=0, max_features=None, strip_accents='unicode',lowercase =True,
        analyzer='word', token_pattern=r'\w{3,}', ngram_range=(1,1), sublinear_tf=True, stop_words = "english")
        transformedMessage=tfv.fit_transform(message)
        return transformedMessage

In [13]:
newdf.TAGS.unique()

array(['GREETINGS', 'NAME', 'ACCOUNT_NUMBER', 'ACCOUNT_DETAILS_VALIDITY',
       'ACCOUNT_DETAILS_INVALIDITY', 'ACCOUNT_DOES_NOT_EXIST', 'INVEST',
       'STOCK_SUGGESTIONS', 'MORE_HELP', 'THANKS', 'EXIT'], dtype=object)

In [14]:
newdf.TAGS = pd.Categorical(newdf.TAGS)
newdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74 entries, 0 to 73
Data columns (total 2 columns):
TAGS        74 non-null category
PATTERNS    74 non-null object
dtypes: category(1), object(1)
memory usage: 4.1+ KB


In [15]:
le = LabelEncoder()
newdf['Target'] = le.fit_transform(newdf.TAGS)
newdf

,TAGS,PATTERNS,Target
0,GREETINGS,Hi Hello,5
1,GREETINGS,How are you,5
2,GREETINGS,Is there anyone,5
3,GREETINGS,Good Morning,5
4,GREETINGS,I need some help,5
5,GREETINGS,Howdy !!,5
6,GREETINGS,Hey,5
7,GREETINGS,I have a question regarding investments,5
8,GREETINGS,hi,5
9,GREETINGS,Good Afternoon ! How is your day going?,5


In [16]:
processed_set = []
only_text = pd.DataFrame(newdf['PATTERNS'])
for i in range(0, len(newdf)):
    processed = PreProcessMessage().preprocess_message(only_text['PATTERNS'][i])
    processed_set.append(processed)

In [17]:
X_train, X_test, y_train, y_test = train_test_split(processed_set,newdf.Target, test_size=0.2, random_state=42)

In [18]:
tfv=TfidfVectorizer(min_df=0, max_features=None, strip_accents='unicode',lowercase =True,
        analyzer='word', token_pattern=r'\w{3,}', ngram_range=(1,1), sublinear_tf=True, stop_words = "english")
X_train_transformed=tfv.fit_transform(X_train)
X_test_transformed=tfv.transform(X_test)

In [69]:
model=LogisticRegression(multi_class='ovr')
model.fit(X_train_transformed,y_train)
model.score(X_train_transformed, y_train)

0.7288135593220338

In [70]:
model.score(X_test_transformed, y_test)

0.2

## Neural Nets

In [35]:
with open(os.getcwd()+'/../Data/convertcsv.json') as file:
    data = json.load(file)
words = []
labels = []
docs_x = []
docs_y = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])
        
    if intent['tag'] not in labels:
        labels.append(intent['tag'])

stemmer = LancasterStemmer()
words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

labels = sorted(labels)

training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []

    wrds = [stemmer.stem(w.lower()) for w in doc]

    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)


training = np.array(training)
output = np.array(output)

In [36]:
tf.reset_default_graph()
net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)
model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)

Training Step: 5999  | total loss: 0.25710 | time: 0.021s
| Adam | epoch: 1000 | loss: 0.25710 - acc: 0.9857 -- iter: 40/43
Training Step: 6000  | total loss: 0.23315 | time: 0.024s
| Adam | epoch: 1000 | loss: 0.23315 - acc: 0.9871 -- iter: 43/43
--


In [45]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return np.array(bag)

results = model.predict([bag_of_words("Gaurang is my first name", words)])
results_index = np.argmax(results)
tag = labels[results_index]
print(tag)

name
